<a href="https://colab.research.google.com/github/BrianDL/fisica_computacional/blob/main/2%20-%20Perihelio%20de%20Mercurio/simulacion_mercurio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulación de la Órbita de Mercurio

Este notebook contiene la implementación del método de Runge-Kutta de segundo orden (RK2) para simular la órbita de Mercurio y calcular la precesión de su perihelio.


In [1]:
import numpy as np
import matplotlib.pyplot as plt

def simular_orbita_mercurio(
        x_inicial:float = None,  ### hacemos esta configuración más abajo
        y_inicial:float = 0,
        vx_inicial:float = 0,
        vy_inicial:float = None, ### hacemos esta configuración más abajo
        iteraciones:int = 20000,
        delta_t: float = 0.0001,
        alpha: float = 0.0008
    ):

    ### Configurando los valores iniciales por defecto
    # Constantes
    G = 6.67430e-11  # Constante gravitacional (m^3 kg^-1 s^-2)
    M_s = 1.989e30   # Masa del Sol (kg)

    a = 0.39
    e = 0.206

    # Inicializar variables
    x = x_inicial if x_inicial else (1+e)*a
    y = y_inicial
    vx = vx_inicial
    vy = vy_inicial if vy_inicial else np.sqrt( G*M_s*(1-e)/( a*(1+e) ) )
    t = 0  # Tiempo inicial siempre igual cero

    # Listas para almacenar resultados
    tiempos = [t]
    posiciones = [(x, y)]
    velocidades = [(vx, vy)]

    # Bucle principal de la simulación
    for i in range(iteraciones):
        # Calcular variables radiales
        r = np.sqrt(x**2 + y**2)
        v = np.sqrt(vx**2 + vy**2)

        # Calcular los valores intermedios (k1)
        k1_x = delta_t * vx
        k1_y = delta_t * vy
        k1_vx = delta_t * (-G * M_s * x / r**3) * (1 + alpha / r**2)
        k1_vy = delta_t * (-G * M_s * y / r**3) * (1 + alpha / r**2)

        # Calcular los valores finales (k2)
        x_mid = x + k1_x/2
        y_mid = y + k1_y/2
        vx_mid = vx + k1_vx/2
        vy_mid = vy + k1_vy/2
        r_mid = np.sqrt(x_mid**2 + y_mid**2)

        k2_x = delta_t * vx_mid
        k2_y = delta_t * vy_mid
        k2_vx = delta_t * (-G * M_s * x_mid / r_mid**3) * (1 + alpha / r_mid**2)
        k2_vy = delta_t * (-G * M_s * y_mid / r_mid**3) * (1 + alpha / r_mid**2)

        # Actualizar los valores para el siguiente paso
        x += k2_x
        y += k2_y
        vx += k2_vx
        vy += k2_vy
        t += delta_t

        # Guardar los resultados en cada paso
        tiempos.append(t)
        posiciones.append((x, y))
        velocidades.append((vx, vy))

    return tiempos, posiciones, velocidades

Esta función `simular_orbita_mercurio` implementa el método RK2 para simular la órbita de Mercurio. En las siguientes celdas, añadiremos código para ejecutar la simulación, visualizar los resultados y calcular la precesión del perihelio.

In [ ]:
import math

def calcular_precesion(alpha):
    # Ejecutar la simulación
    tiempos, posiciones, velocidades = simular_orbita_mercurio(
        alpha=alpha
    )

    # Encontrar los perihelios
    distancias = [math.sqrt(x**2 + y**2) for x, y in posiciones]
    perihelios = []
    for i in range(1, len(distancias) - 1):
        if distancias[i] < distancias[i-1] and distancias[i] < distancias[i+1]:
            perihelios.append(i)

    # Calcular los ángulos de los perihelios
    angulos_perihelio = []
    for i in perihelios:
        x, y = posiciones[i]
        angulo = math.atan2(y, x)
        if angulo < 0:
            angulo += 2 * math.pi
        angulos_perihelio.append(angulo)

    # Calcular la precesión
    precesiones = []
    for i in range(1, len(angulos_perihelio)):
        precesion = angulos_perihelio[i] - angulos_perihelio[i-1]
        if precesion < 0:
            precesion += 2 * math.pi
        precesiones.append(precesion)

    # Calcular la precesión promedio por órbita
    precesion_promedio = sum(precesiones) / len(precesiones)

    # Convertir la precesión a segundos de arco por siglo
    periodo_orbital_mercurio = 87.97 * 24 * 3600  # en segundos
    orbitas_por_siglo = (100 * 365.25 * 24 * 3600) / periodo_orbital_mercurio
    precesion_por_siglo = precesion_promedio * orbitas_por_siglo
    precesion_segundos_arco = precesion_por_siglo * 180 / math.pi * 3600

    return precesion_segundos_arco

# Ejemplo de uso
alpha_test = 0.0008
precesion = calcular_precesion(alpha_test)
print(f"Para alpha = {alpha_test}, la precesión es de {precesion:.2f} segundos de arco por siglo.")